In [1]:
import json
import re
import collections
import numpy as np
import multiprocessing

import warnings
warnings.filterwarnings('ignore')

import wikipedia
import pandas as pd

wikipedia.set_lang('en')

from fdh_gallica import Search

N_PROC = 8

In [2]:
df = pd.read_pickle('named_subject.pkl')

In [3]:
df.named_subject.unique().shape

(1937,)

### Wikipedia Search

In [14]:
all_names = df.named_subject.unique().tolist()[0:500]

In [5]:
all_names

['Auguez, Mathilde',
 'Bruant, Aristide',
 'Brasseur, Albert',
 'Decroza, Francine',
 'Lender, Marcelle',
 'Bernhardt, Sarah',
 'Regnard, Victor-Auguste',
 'Darcourt, Juliette',
 'Dauville, Louise',
 'Gobin, Charles Constant',
 'Nâser ed-Din',
 'Gille, Philippe',
 'Gélabert, Conchita, Marie',
 'Joncières, Victorin',
 'Detaille, Édouard',
 "Auriac, Eugène d'",
 'Terront, Charles',
 'Georges, Édouard',
 'Armand, Victorin',
 'Bellot',
 'Blanc, Louis',
 'Lugné-Poë',
 'Faivre, Abel',
 'Millière, Joséphine Louise',
 'Hading, Jane',
 'Joséphine',
 'Poubelle, Eugène-René',
 'Bastide, Jules',
 'Cassive',
 'Offenbach, Jacques',
 'Nadar',
 'Chevreul, Eugène',
 'Nadar, Paul',
 'Chevreul, Henri',
 'Berlioz, Hector',
 'Gallifet, Gaston Alexandre Auguste de',
 'Clairville',
 'Crémieux, Adolphe',
 'Sandeau, Jules',
 'Méry, Joseph',
 'Courbet, Gustave',
 'Janin, Jules',
 'Crétineau-Joly, Jacques',
 'Gautier, Théophile',
 'Veuillot, Louis',
 'Féval, Paul',
 'Karr, Alphonse',
 'Feuillet, Octave',
 'Champ

In [6]:
pool = multiprocessing.Pool(processes=N_PROC)
# wiki_searchs = dict(zip(all_names, pool.map(hacked_search, all_names)))
# with open('wiki_searchs.json', 'w') as fp:
#     json.dump(wiki_searchs, fp)
# print(len(wiki_searchs))

In [7]:
def hacked_page(x):
    try:
        page = wikipedia.page(x)
    except:
        page = ''
    return page

def hacked_content(x):
    try:
        page = wikipedia.page(x).content
    except:
        page = ''
    return page
    

#### First try searching page summaries

In [8]:
# copied from extract-wiki-info.ipunb

def hacked_summary(x):
    try:
        s = wikipedia.summary(x, sentences=2)
        return s
    except:
        return ''

In [15]:
#wiki_summaries = dict(zip(all_names, pool.map(hacked_summary, all_names)))
wiki_summaries = dict(zip(all_names,map(hacked_summary, all_names)))

save name-wiki summary association in json

In [16]:
with open('wiki_summaries.json', 'w') as fp:
    json.dump(wiki_summaries, fp)

dataframe of named subjects 

In [17]:
wiki_frame = pd.DataFrame(list(wiki_summaries.items()), columns=['name','summary'])

In [19]:
wiki_frame[wiki_frame.summary==''].shape

(171, 2)

In [20]:
wiki_frame.shape

(500, 2)

In [13]:
#wikipedia.set_lang('fr')
#wiki_summaries_fr = dict(zip(all_names,map(hacked_summary, all_names)))
#with open('wiki_summaries_fr.json', 'w') as fp:
#    json.dump(wiki_summaries_fr, fp)

### BnF Data Search